# Analisando a verticalização de uma região pelo IPTU 

Neste NoteBook vamos demonstrar como extrair informações de relavência para a compreensão das dinêmicas da cidade com a base de dados de IPTU espacializando a informação.

Nesse caso vamos analisar a verticalização por quadra fiscal em uma determinada sub, gerando gráficos e mapas

In [1]:
import pandas as pd

df = pd.read_csv("./downloads/IPTU_2020.csv", error_bad_lines=False, encoding='iso-8859-9', sep=';')
df.head()

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,TIPO DE CONTRIBUINTE 1,CPF/CNPJ DO CONTRIBUINTE 1,NOME DO CONTRIBUINTE 1,TIPO DE CONTRIBUINTE 2,CPF/CNPJ DO CONTRIBUINTE 2,NOME DO CONTRIBUINTE 2,...,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE
0,0010030001-4,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,1924,1,"13,00",Loja,Comercial horizontal - padrão B,De esquina,"0,20",1963,1,0
1,0010030002-2,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,1944,1,"6,00",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0
2,0010030003-0,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,1965,2,"7,85",Loja,Comercial horizontal - padrão B,Normal,"0,30",1963,1,0
3,0010030004-9,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX2094XXXX,AUGUSTO CESAR DE MATTOS JUNIOR,NaN,,NaN,...,1944,1,"6,05",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0
4,0010030005-7,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX2094XXXX,AUGUSTO CESAR DE MATTOS JUNIOR,NaN,,NaN,...,1944,1,"6,70",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0


## Colunas relevantes

Para compreender a verticalização temos dois dados importantes, o `ANO DA CONSTRUCAO CORRIGIDO` e a `QUANTIDADE DE PAVIMENTOS`

In [2]:
list(df.columns)

['NUMERO DO CONTRIBUINTE',
 'ANO DO EXERCICIO',
 'NUMERO DA NL',
 'DATA DO CADASTRAMENTO',
 'TIPO DE CONTRIBUINTE 1',
 'CPF/CNPJ DO CONTRIBUINTE 1',
 'NOME DO CONTRIBUINTE 1',
 'TIPO DE CONTRIBUINTE 2',
 'CPF/CNPJ DO CONTRIBUINTE 2',
 'NOME DO CONTRIBUINTE 2',
 'NUMERO DO CONDOMINIO',
 'CODLOG DO IMOVEL',
 'NOME DE LOGRADOURO DO IMOVEL',
 'NUMERO DO IMOVEL',
 'COMPLEMENTO DO IMOVEL',
 'BAIRRO DO IMOVEL',
 'REFERENCIA DO IMOVEL',
 'CEP DO IMOVEL',
 'QUANTIDADE DE ESQUINAS/FRENTES',
 'FRACAO IDEAL',
 'AREA DO TERRENO',
 'AREA CONSTRUIDA',
 'AREA OCUPADA',
 'VALOR DO M2 DO TERRENO',
 'VALOR DO M2 DE CONSTRUCAO',
 'ANO DA CONSTRUCAO CORRIGIDO',
 'QUANTIDADE DE PAVIMENTOS',
 'TESTADA PARA CALCULO',
 'TIPO DE USO DO IMOVEL',
 'TIPO DE PADRAO DA CONSTRUCAO',
 'TIPO DE TERRENO',
 'FATOR DE OBSOLESCENCIA',
 'ANO DE INICIO DA VIDA DO CONTRIBUINTE',
 'MES DE INICIO DA VIDA DO CONTRIBUINTE',
 'FASE DO CONTRIBUINTE']

In [3]:
df[['NUMERO DO CONTRIBUINTE', 'ANO DA CONSTRUCAO CORRIGIDO', 'QUANTIDADE DE PAVIMENTOS', 'NUMERO DO CONDOMINIO']]

,NUMERO DO CONTRIBUINTE,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,NUMERO DO CONDOMINIO
0,0010030001-4,1924,1,00-0
1,0010030002-2,1944,1,00-0
2,0010030003-0,1965,2,00-0
3,0010030004-9,1944,1,00-0
4,0010030005-7,1944,1,00-0
...,...,...,...,...
3498639,3101180238-1,1978,5,01-9
3498640,3101180239-1,1978,5,01-9
3498641,3101180240-3,1978,5,01-9
3498642,3101190001-4,0,0,00-0


## Localizando e agrupando os registros

O número do contribuinte contem uma informação espacial importante, pois por ele podemos localizar o Setor, a Quadra e o Lote e a partir dai relacionar com as informações geográficas dessas feições.

Portanto vamos separ essas informações e agrupa-las por lote, uma vez que temos lotes condominiais que possuem mais do que um contribuinte.

In [4]:
df['setor'] = df['NUMERO DO CONTRIBUINTE'].str[:3]

In [5]:
df['quadra'] = df['NUMERO DO CONTRIBUINTE'].str[3:6]

### Lotes condominiais

É necessário tratar os lotes que são condominiais e tem mais do que um número de contribuinte.

Para isso vamos desiignar as letras CD e o `NUMERO DO CONDOMINIO` para qualifica-los

In [19]:
%%time
# Essa célula costuma demorar alguns minutos para rodar
# Seria interessante considerar uma agregação somente com os lotes condominiais antes de roda-la

def lote(x):
    if x[1]['NUMERO DO CONDOMINIO'] == '00-0':
        return x[1]['NUMERO DO CONTRIBUINTE'][6:10]
    else:
        return 'CD' + x[1]['NUMERO DO CONDOMINIO'][:2]
    
df['lote'] = list(map(lambda x: lote(x), df[['NUMERO DO CONTRIBUINTE', 'NUMERO DO CONDOMINIO']].iterrows()))

In [21]:
%%time
(df[['setor', 'quadra', 'lote', 'ANO DA CONSTRUCAO CORRIGIDO', 'QUANTIDADE DE PAVIMENTOS']].
 groupby(['setor', 'quadra', 'lote']).
 count())

CPU times: user 797 ms, sys: 112 ms, total: 909 ms
Wall time: 908 ms


ANO DA CONSTRUCAO CORRIGIDO  QUANTIDADE DE PAVIMENTOS
setor quadra lote                                                       
001   003    0001                            1                         1
             0002                            1                         1
             0003                            1                         1
             0004                            1                         1
             0005                            1                         1
...                                        ...                       ...
310   117    CD01                           44                        44
             CD02                          135                       135
      118    CD01                          240                       240
      119    0001                            1                         1
      120    0001                            1                         1

[1661100 rows x 2 columns]

In [22]:
df

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,TIPO DE CONTRIBUINTE 1,CPF/CNPJ DO CONTRIBUINTE 1,NOME DO CONTRIBUINTE 1,TIPO DE CONTRIBUINTE 2,CPF/CNPJ DO CONTRIBUINTE 2,NOME DO CONTRIBUINTE 2,...,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE,setor,quadra,lote
0,0010030001-4,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,Loja,Comercial horizontal - padrão B,De esquina,"0,20",1963,1,0,001,003,0001
1,0010030002-2,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0,001,003,0002
2,0010030003-0,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,Loja,Comercial horizontal - padrão B,Normal,"0,30",1963,1,0,001,003,0003
3,0010030004-9,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX2094XXXX,AUGUSTO CESAR DE MATTOS JUNIOR,NaN,,NaN,...,Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0,001,003,0004
4,0010030005-7,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX2094XXXX,AUGUSTO CESAR DE MATTOS JUNIOR,NaN,,NaN,...,Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0,001,003,0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498639,3101180238-1,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX5127XXXX,CECILIA OTUKA,NaN,,NaN,...,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,"0,20",1985,1,0,310,118,CD01
3498640,3101180239-1,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX6436XXXX,JOSE ESTANISLAU CAMPOS MACHADO FILHO,NaN,,NaN,...,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,"0,20",1985,1,0,310,118,CD01
3498641,3101180240-3,2020,1,11/01/20,PESSOA FISICA (CPF),XXXXXX6473XXXX,DOUGLAS MACEDO,PESSOA FISICA (CPF),XXXXXX9655XXXX,JOICE FERNANDA PAIXAO MACEDO,...,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,"0,20",1985,1,0,310,118,CD01
3498642,3101190001-4,2020,1,11/01/20,PESSOA JURIDICA (CNPJ),XXXXXXXXXXXXXX,ESTADO DE SAO PAULO,NaN,,NaN,...,Terreno,TERRENO,De esquina,"0,00",1988,1,0,310,119,0001
